In [1]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.utils.video_visualizer import VideoVisualizer

D:\Ott_diploma_proj_prog_files\conda\envs\ott_diploma\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classes = ['icy_tail', 'icy_asphalt', 'powdery_snow', 'snow_drift', 'puffy_road_snow', 'snowy_road']

MetadataCatalog.get("category").set(thing_classes=classes, thing_colors = [
        (255,255,0),(0,0,255),(0,255,0),(255,0,255), (180,165,0), (187,132,156)]) #Done by me
  
    

microcontroller_metadata = MetadataCatalog.get("category")

In [3]:
cfg_instance_seg = get_cfg()
cfg_instance_seg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg_instance_seg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
cfg_instance_seg.MODEL.WEIGHTS = os.path.join("output", "model_final.pth")
cfg_instance_seg.MODEL.ROI_HEADS.NUM_CLASSES = 6
predictor = DefaultPredictor(cfg_instance_seg)

[05/04 13:58:08 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from output\model_final.pth ...


In [4]:
class MyVisualizer(Visualizer):
    def _jitter(self, color ):
        return color

In [5]:
def new_output(outputs):
    new_output = detectron2.structures.instances.Instances(image_size = (1080, 1920))
    new_output.set('pred_classes', outputs['instances'].pred_classes)
    new_output.set('pred_masks', outputs['instances'].pred_masks)
    return new_output

In [ ]:
#with filming
cap = cv2.VideoCapture("DATA/video/7.mp4")
result = cv2.VideoWriter('ready_vids/before_optim/vid7.avi', cv2.VideoWriter_fourcc(*"MJPG"), 20.0, (1536, 864))
while(cap.isOpened()):
    success, img = cap.read()
    if success == True:
        img = cv2.resize(img, (1920, 1080), interpolation = cv2.INTER_AREA)
        outputs = predictor(img)
        outputs = new_output(outputs)
        v = MyVisualizer(img[:, :, ::-1],
                        metadata=microcontroller_metadata, 
                        scale=0.8, 
                        instance_mode=ColorMode.SEGMENTATION, # removes the colors of unsegmented pixels
        )
        v = v.draw_instance_predictions(outputs.to("cpu"))
        cv2.imshow("Image!", v.get_image()[:, :, ::-1])
        
        #print(v.get_image()[:, :, ::-1].shape)
        result.write(v.get_image()[:, :, ::-1])
        
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    else:
        break
    
result.release()
cap.release()
cv2.destroyAllWindows()
print("Done, Don!")

In [6]:
#with no filming
cap = cv2.VideoCapture("DATA/video/1.mp4")
while(cap.isOpened()):
    success, img = cap.read()
    if success == True:
        img = cv2.resize(img, (1920, 1080), interpolation = cv2.INTER_AREA)
        outputs = predictor(img)
        
        v = MyVisualizer(img[:, :, ::-1],
                        metadata=microcontroller_metadata, 
                        scale=0.8, 
                        instance_mode=ColorMode.SEGMENTATION, # removes the colors of unsegmented pixels
        )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        cv2.imshow("Image!", v.get_image()[:, :, ::-1])
        
        #print(v.get_image()[:, :, ::-1].shape)
        
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
print("Done, Don!")

Done, Don!
